In [8]:
 import os

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd
import tensorflow as tf

from skimage.io import imread
from skimage.transform import resize
from PIL import Image
from imgaug import augmenters as iaa

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from keras.utils import np_utils

from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras import Model

In [9]:
train_dir = "C:/Users/dedsec995/Projects/tensorflow/breed_dog_only/Data/dog/images"

In [20]:
inp = Input((224, 224, 3))
backbone = DenseNet121(input_tensor=inp,
                       weights="imagenet",
                       include_top=False)
x = backbone.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
outp = Dense(120, activation="softmax")(x)

model = Model(inp, outp)

In [21]:
# model.summary()

In [22]:
for layer in model.layers[:-6]:
    layer.trainable = False

In [23]:
# model.summary()

In [24]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics = ["accuracy"])

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator(
                                   rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode='nearest',
                                   validation_split=0.1
)
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size  = 64,
                                                    color_mode = 'rgb',
                                                    class_mode="categorical",
                                                    target_size = (224, 224),
                                                    subset='training')     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  train_datagen.flow_from_directory( train_dir,
                                                          batch_size  = 64, 
                                                          color_mode = 'rgb',
                                                          class_mode="categorical",
                                                          target_size = (224, 224),
                                                          subset='validation')

def my_gen(gen):
    while True:
        try:
            data, labels = next(gen)
            yield data, labels
        except:
            pass

Found 18571 images belonging to 120 classes.
Found 2009 images belonging to 120 classes.


In [ ]:
history = model.fit(train_generator,
                              validation_data=validation_generator, 
                              steps_per_epoch=len(train_generator), 
                              validation_steps=len(validation_generator),
                              epochs=60)

Epoch 1/60
291/291 [==============================] - 231s 794ms/step - loss: 3.0085 - accuracy: 0.2725 - val_loss: 1.5863 - val_accuracy: 0.5734
Epoch 2/60
291/291 [==============================] - 234s 804ms/step - loss: 1.8345 - accuracy: 0.4864 - val_loss: 1.3661 - val_accuracy: 0.6167
Epoch 3/60
291/291 [==============================] - 231s 795ms/step - loss: 1.6300 - accuracy: 0.5351 - val_loss: 1.3095 - val_accuracy: 0.6212
Epoch 4/60
291/291 [==============================] - 235s 806ms/step - loss: 1.5271 - accuracy: 0.5579 - val_loss: 1.2659 - val_accuracy: 0.6297
Epoch 5/60
291/291 [==============================] - 230s 790ms/step - loss: 1.4444 - accuracy: 0.5849 - val_loss: 1.2643 - val_accuracy: 0.6322
Epoch 6/60
291/291 [==============================] - 224s 771ms/step - loss: 1.4323 - accuracy: 0.5834 - val_loss: 1.2379 - val_accuracy: 0.6317
Epoch 7/60
291/291 [==============================] - 224s 771ms/step - loss: 1.3863 - accuracy: 0.5950 - val_loss: 1.2119 -

In [ ]:
plt.rcParams['figure.figsize'] = (6,6)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()

plt.show()

In [ ]:
for layer in model.layers[:]:
    layer.trainable = True

In [ ]:
# a check point callback to save our best weights
checkpoint = ModelCheckpoint('dog_breed_classifier_model_my.h5', 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max', 
                             save_weights_only=True)

# a reducing lr callback to reduce lr when val_loss doesn't increase
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                   patience=1, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=2, min_lr=1e-7)

# for early stop
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=5)

In [ ]:
history = model.fit_generator(train_generator,
                              validation_data=validation_generator, 
#                               steps_per_epoch=len(train_generator)/20, 
#                               validation_steps=len(val_gen),
                              epochs=60,
                              callbacks=[checkpoint, reduce_lr, early_stop])

In [ ]:
plt.rcParams['figure.figsize'] = (6,6)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()

plt.show()

In [ ]:
print(max(val_acc))

In [ ]:
from tensorflow.keras.preprocessing import image
path = 'C:/Users/dedsec995/Projects/tensorflow/breed_dog_only/notebook/test_1.jpg'
img = image.load_img(path, target_size=(224, 224))
x = image.img_to_array(img)
print(x)
# print(x.ndim)
x = np.expand_dims(x, axis=0)
# print(x.ndim)
x = x/255.
print(x)
classes = model.predict(x)
# print(fn)
print(classes)
print(classes[0][74])

In [22]:
model.save('my_method_dog_only')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_method_dog_only\assets


In [25]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)